# Your First RAQA Application WITH PDF


Let's look at a rather complicated looking visual representation of a basic RAQA application.

<img src="https://i.imgur.com/PvlaIUO.png" width='1000em'/>

### Install Dependencies

In [1]:
!pip install -q -U numpy matplotlib plotly pandas scipy scikit-learn openai python-dotenv

### Import Dependencies

In [35]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

import nest_asyncio
nest_asyncio.apply()

In [36]:
# OPEN AI stuff

import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### ASSIGNMENT QUESTION 1
#### Allow it to work with PDF files


In [37]:
# Install pdf dependencies
# !pip install -q -U pymupdf

import fitz  # PyMuPDF

class PDFLoader:
    def __init__(self, path):
        self.path = path
        self.documents = []

    def load_documents(self):  # Renamed from load_file to load_documents
        with fitz.open(self.path) as doc:
            text = ""
            for page in doc:
                text += page.get_text()
            self.documents.append(text)
        return self.documents  # Make sure to return the documents after loading


# Integrate pdf loading into the app
            
def load_documents(file_path):
    if file_path.endswith('.pdf'):
        loader = PDFLoader(file_path)
    else:
        loader = TextFileLoader(file_path)
    documents = loader.load_documents()
    return documents


In [38]:

# Load the pdf
pdf_documents = load_documents("data/King_Lear.pdf")
print(pdf_documents[0][:100])  # Print the first 100 characters of the extracted text


ACT I
SCENE I. King Lear's palace.
Enter KENT, GLOUCESTER, and EDMUND
KENT
I thought the king had mo


#### Split Text Into Chunks

In [39]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(pdf_documents)
len(split_documents)

189

In [40]:
split_documents[1]

"g so proper.\nGLOUCESTER\nBut I have, sir, a son by order of law, some year\nelder than this, who yet is no dearer in my account:\nthough this knave came something saucily into the\nworld before he was sent for, yet was his mother\nfair; there was good sport at his making, and the\nwhoreson must be acknowledged. Do you know this\nnoble gentleman, Edmund?\nEDMUND\nNo, my lord.\nGLOUCESTER\nMy lord of Kent: remember him hereafter as my\nhonourable friend.\nEDMUND\nMy services to your lordship.\nKENT\nI must love you, and sue to know you better.\nEDMUND\nSir, I shall study deserving.\nGLOUCESTER\nHe hath been out nine years, and away he shall\nagain. The king is coming.\nSennet. Enter KING LEAR, CORNWALL, ALBANY, GONERIL, REGAN, CORDELIA, and\nAttendants\nKING LEAR\nAttend the lords of France and Burgundy, Gloucester.\nGLOUCESTER\nI shall, my liege.\nExeunt GLOUCESTER and EDMUND\nKING LEAR\nMeantime we shall express our darker purpose.\nGive me the map there. Know that we have divided\nI

In [41]:
split_documents[0:1]

["ACT I\nSCENE I. King Lear's palace.\nEnter KENT, GLOUCESTER, and EDMUND\nKENT\nI thought the king had more affected the Duke of\nAlbany than Cornwall.\nGLOUCESTER\nIt did always seem so to us: but now, in the\ndivision of the kingdom, it appears not which of\nthe dukes he values most; for equalities are so\nweighed, that curiosity in neither can make choice\nof either's moiety.\nKENT\nIs not this your son, my lord?\nGLOUCESTER\nHis breeding, sir, hath been at my charge: I have\nso often blushed to acknowledge him, that now I am\nbrazed to it.\nKENT\nI cannot conceive you.\nGLOUCESTER\nSir, this young fellow's mother could: whereupon\nshe grew round-wombed, and had, indeed, sir, a son\nfor her cradle ere she had a husband for her bed.\nDo you smell a fault?\nKENT\nI cannot wish the fault undone, the issue of it\nbeing so proper.\nGLOUCESTER\nBut I have, sir, a son by order of law, some year\nelder than this, who yet is no dearer in my account:\nthough this knave came something saucily

### Embeddings and Vectors


In [42]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

In [43]:
vector_db.search_by_text("Your servant Kent. Where is your servant Caius?", k=3)

[("stay a little. Ha!\nWhat is't thou say'st? Her voice was ever soft,\nGentle, and low, an excellent thing in woman.\nI kill'd the slave that was a-hanging thee.\nCaptain\n'Tis true, my lords, he did.\nKING LEAR\nDid I not, fellow?\nI have seen the day, with my good biting falchion\nI would have made them skip: I am old now,\nAnd these same crosses spoil me. Who are you?\nMine eyes are not o' the best: I'll tell you straight.\nKENT\nIf fortune brag of two she loved and hated,\nOne of them we behold.\nKING LEAR\nThis is a dull sight. Are you not Kent?\nKENT\nThe same,\nYour servant Kent: Where is your servant Caius?\nKING LEAR\nHe's a good fellow, I can tell you that;\nHe'll strike, and quickly too: he's dead and rotten.\nKENT\nNo, my good lord; I am the very man,--\nKING LEAR\nI'll see that straight.\nKENT\nThat, from your first of difference and decay,\nHave follow'd your sad steps.\nKING LEAR\nYou are welcome hither.\nKENT\nNor no man else: all's cheerless, dark, and deadly.\nYour e

## Prompts
#### Creating and Prompting OpenAI's `gpt-3.5-turbo`!


In [46]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    user_role_prompt.create_message(
        content="What is the best way to make a morning coffee? provide answer in list"
    ),
    system_role_prompt.create_message(expertise="Coffee"),
]

response = chat_openai.run(messages)

In [47]:
print(response)

1. Start by grinding your coffee beans fresh for optimal flavor.

2. Boil water to the desired temperature, typically around 200°F (93°C).

3. Add the freshly ground coffee to a filter in your coffee maker or pour-over brewer.

4. Slowly pour the hot water over the coffee grounds in a circular motion, ensuring all grounds are saturated.

5. Allow the coffee to brew for the recommended time, typically 3-4 minutes.

6. Once brewed, pour the coffee into your favorite mug and add any desired milk, cream, or sweeteners.

7. Take a moment to savor the aroma before taking your first sip and enjoy your perfect morning coffee.


### Retrieval Augmented Question Answering Prompt


In [48]:
RAQA_PROMPT_TEMPLATE = """ \
Use the provided context to answer the user's query. 

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

raqa_prompt = SystemRolePrompt(RAQA_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""


user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)
        
        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = raqa_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)
        
        return self.llm.run([formatted_system_prompt, formatted_user_prompt])

In [50]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [53]:
retrieval_augmented_qa_pipeline.run_pipeline("Who is King Lear? provide answer in a bullet points")

'- King Lear is a character in William Shakespeare\'s play titled "King Lear."\n- Lear is depicted as an aging monarch who decides to divide his kingdom among his three daughters, testing their love for him through flattery.\n- The play delves into themes of power, family relationships, betrayal, madness, and the consequences of arrogance.'

### Visibility Tooling


In [60]:
### install dependencies
!pip install -q -U wandb

In [61]:
# wandb keys via https://wandb.ai/authorize

wandb_key = getpass("Weights and Biases API Key: ")
os.environ["WANDB_API_KEY"] = wandb_key

In [62]:
# Run the visibility tracer

import wandb

os.environ["WANDB_NOTEBOOK_NAME"] = "Python RAQA Example with PDF.ipynb"
wandb.init(project="Visibility Example using PDF")

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [63]:
import datetime
from wandb.sdk.data_types.trace_tree import Trace

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase, wandb_project = None) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.wandb_project = wandb_project

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)
        
        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = raqa_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        
        start_time = datetime.datetime.now().timestamp() * 1000

        try:
            openai_response = self.llm.run([formatted_system_prompt, formatted_user_prompt], text_only=False)
            end_time = datetime.datetime.now().timestamp() * 1000
            status = "success"
            status_message = (None, )
            response_text = openai_response.choices[0].message.content
            token_usage = dict(openai_response.usage)
            model = openai_response.model

        except Exception as e:
            end_time = datetime.datetime.now().timestamp() * 1000
            status = "error"
            status_message = str(e)
            response_text = ""
            token_usage = {}
            model = ""

        if self.wandb_project:
            root_span = Trace(
                name="root_span",
                kind="llm",
                status_code=status,
                status_message=status_message,
                start_time_ms=start_time,
                end_time_ms=end_time,
                metadata={
                    "token_usage" : token_usage,
                    "model_name" : model
                },
                inputs= {"system_prompt" : formatted_system_prompt, "user_prompt" : formatted_user_prompt},
                outputs= {"response" : response_text}
            )

            root_span.log(name="openai_trace")
        
        return response_text if response_text else "We ran into an error. Please try again later. Full Error Message: " + status_message

In [64]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    wandb_project="LLM Visibility Example using PDF"
)

In [65]:
retrieval_augmented_qa_pipeline.run_pipeline("Who is Batman?") # should return I dont know

"I don't know."

In [70]:
retrieval_augmented_qa_pipeline.run_pipeline("What happens to Cordelia?")

"Cordelia is awarded to the King of France as his wife after Burgundy rejects her due to King Lear's reduced offer of dowry. The King of France admires Cordelia's virtues and accepts her despite her lack of dowry. Cordelia then leaves with the King of France, leaving behind her sisters Goneril and Regan. At the end of the scene, Cordelia bids farewell to her sisters and expresses her wishes for their father's well-being."

In [67]:
retrieval_augmented_qa_pipeline.run_pipeline("What happens to Cordelia at the end?")

"Cordelia, at the end of the text provided, is faced with rejection from her father, King Lear. The King disinherits her for not indulging in flattery like her sisters Goneril and Regan. Despite the King's harsh treatment, the King of France steps in and appreciates Cordelia for who she is, marrying her and declaring her as the Queen of France. Cordelia acknowledges this and bids farewell to her sisters and her father, choosing to start a new life with the King of France."

In [68]:
retrieval_augmented_qa_pipeline.run_pipeline("Does Cordelia die at the end?")

'Yes, Cordelia dies at the end of the passage provided. King Lear enters holding Cordelia dead in his arms, expressing deep sorrow and lamenting her death.'

In [71]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
